In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import warnings
from IPython.display import clear_output
from multiprocessing import Pool
from time import time
warnings.filterwarnings('ignore')

In [2]:
beijing_aq_plus_met = pd.read_csv('tmp_beijing_aq_plus_met_preprocessed.csv.gz')
beijing_aq_plus_met.head(2)

,time,station_id,longitude,latitude,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01 00:00:00,1001,116.173553,40.090679,138.0,159.4,56.3,0.9,50.8,17.2,0.0,20.0,56.0,7.92,13.0
1,2014-05-01 00:00:00,1002,116.205310,40.003950,89.0,132.9,30.5,0.8,96.5,7.6,0.0,20.0,56.0,7.92,13.0


In [3]:
beijing_aq_plus_met['time'] = pd.to_datetime(beijing_aq_plus_met['time'])
beijing_aq_plus_met2 = beijing_aq_plus_met.sort_values(by=['time', 'station_id'])
beijing_aq_plus_met2.head(2)

,time,station_id,longitude,latitude,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01,1001,116.173553,40.090679,138.0,159.4,56.3,0.9,50.8,17.2,0.0,20.0,56.0,7.92,13.0
1,2014-05-01,1002,116.205310,40.003950,89.0,132.9,30.5,0.8,96.5,7.6,0.0,20.0,56.0,7.92,13.0


### Asserting number of data entries per station

In [6]:
for station in beijing_aq_plus_met2.station_id.unique():
    tmp_df = beijing_aq_plus_met2[beijing_aq_plus_met2.station_id==station]
    assert tmp_df.shape == (8760, 15), print(tmp_df.shape)

interpr = ['linear', 'slinear', 'quadratic', 'cubic', 'spline', 'akima']

### Functions

In [7]:
def process_continuous(tmp_df):
    splitter = KFold(n_splits=5, random_state=0, shuffle=True)
    for f_i, (train, test) in enumerate(splitter.split(tmp_df.index)):
        train_index = tmp_df.index[train]
        test_index = tmp_df.index[test]
        
        tmp_df['tmp_data'] = tmp_df[feature]
        tmp_df.loc[test_index, 'tmp_data'] = None
        for algo in interpr:
            filled_vals = tmp_df['tmp_data'].interpolate(method=algo, 
                                                         order=3, 
                                                         limit_direction='both').ffill().bfill()
            tmp_df[algo].loc[test_index] = filled_vals.loc[test_index]
    return tmp_df

def run_k_folds():
    data_df = beijing_aq_plus_met2[['time', 'station_id', feature]]
    for algo in interpr:
        data_df[algo] = None
    
    tmp_df_list = []
    for station in data_df.station_id.unique():
        tmp_df_list.append(data_df[data_df.station_id == station])

    workers = Pool()
    out_list = workers.map(process_continuous, tmp_df_list)
    workers.close()

    final_df = pd.concat(out_list)
    check_df = final_df.dropna(subset=[feature])
    print('###\nRMSE\n###')
    for algo in interpr:
        print(algo, mean_squared_error(check_df[feature], check_df[algo], squared=False))
    print("###")
    return final_df

### Process Continuous variables

In [8]:
init = time()
feature = 'PM25_Concentration'
    
out_df = run_k_folds()
print('time taken:', time()-init, 'seconds')
out_df

###
RMSE
###
linear 16.691215758421183
slinear 16.691215758421187
quadratic 18.293555890378826
cubic 18.66546632451523
spline 18.714090061900002
akima 16.455147907269197
###
time taken: 31.58277678489685 seconds


,time,station_id,PM25_Concentration,linear,slinear,quadratic,cubic,spline,akima,tmp_data
0,2014-05-01 00:00:00,1001,138.0,124.0,124.0,124.0,124.0,144.299167,124.0,NaN
31,2014-05-01 01:00:00,1001,124.0,132.5,132.5,128.758888,121.990135,122.391893,130.841346,124.0
62,2014-05-01 02:00:00,1001,127.0,122.333333,122.333333,116.154103,115.799895,115.851809,118.789354,127.0
93,2014-05-01 03:00:00,1001,129.0,120.666667,120.666667,114.470684,113.949842,114.005624,117.191502,129.0
124,2014-05-01 04:00:00,1001,119.0,127.333333,127.333333,128.984853,128.92544,130.606885,128.652315,119.0
...,...,...,...,...,...,...,...,...,...,...
271435,2015-04-30 19:00:00,1036,77.0,81.25,81.25,78.298068,75.146572,75.506208,78.916295,77.0
271466,2015-04-30 20:00:00,1036,94.0,83.5,83.5,79.56409,72.993144,73.666419,80.717262,94.0
271497,2015-04-30 21:00:00,1036,NaN,91.0,91.0,99.598942,108.946056,104.764411,96.064103,NaN
271528,2015-04-30 22:00:00,1036,88.0,94.0,94.0,94.0,94.0,215.100355,94.0,88.0


In [9]:
init = time()
feature = 'temperature'
    
out_df = run_k_folds()
print('time taken:', time()-init, 'seconds')
out_df

###
RMSE
###
linear 1.2417092080219119
slinear 1.2417092080219119
quadratic 1.3909661561042301
cubic 1.4266029324031693
spline 1.5532791160045132
akima 1.2273974632816096
###
time taken: 2.6284217834472656 seconds


,time,station_id,temperature,linear,slinear,quadratic,cubic,spline,akima,tmp_data
0,2014-05-01 00:00:00,1001,20.0,18.0,18.0,18.0,18.0,23.291469,18.0,NaN
31,2014-05-01 01:00:00,1001,18.0,19.0,19.0,19.075745,19.495432,18.685129,19.0,18.0
62,2014-05-01 02:00:00,1001,18.0,17.666667,17.666667,17.223334,17.418725,16.286484,17.547619,18.0
93,2014-05-01 03:00:00,1001,17.0,17.333333,17.333333,17.410001,17.378087,15.382987,17.412698,17.0
124,2014-05-01 04:00:00,1001,17.0,16.666667,16.666667,16.113088,16.018046,15.134037,16.201058,17.0
...,...,...,...,...,...,...,...,...,...,...
271435,2015-04-30 19:00:00,1036,25.2,26.125,26.125,26.271616,26.447724,25.479128,25.913721,25.2
271466,2015-04-30 20:00:00,1036,25.0,25.55,25.55,26.825671,26.677724,23.941196,25.542528,25.0
271497,2015-04-30 21:00:00,1036,NaN,24.7,24.7,25.459212,25.405055,22.532698,24.743628,NaN
271528,2015-04-30 22:00:00,1036,24.4,21.533333,21.533333,22.438555,23.670347,21.149461,22.068783,24.4


In [10]:
init = time()
feature = 'humidity'
    
out_df = run_k_folds()
print('time taken:', time()-init, 'seconds')
out_df

###
RMSE
###
linear 7.566049355752404
slinear 7.566049355752405
quadratic 9.215021874612322
cubic 9.494899371060727
spline 9.464735340192824
akima 7.616762627181121
###
time taken: 20.488496780395508 seconds


,time,station_id,humidity,linear,slinear,quadratic,cubic,spline,akima,tmp_data
0,2014-05-01 00:00:00,1001,56.0,64.0,64.0,64.0,64.0,56.954709,64.0,NaN
31,2014-05-01 01:00:00,1001,64.0,63.0,63.0,63.930563,63.600047,63.722199,63.75,64.0
62,2014-05-01 02:00:00,1001,70.0,67.666667,67.666667,69.883432,70.060941,68.893182,69.030864,70.0
93,2014-05-01 03:00:00,1001,74.0,71.333333,71.333333,73.616898,73.841411,72.871011,72.737654,74.0
124,2014-05-01 04:00:00,1001,75.0,75.333333,75.333333,76.711483,76.710537,76.808141,76.497354,75.0
...,...,...,...,...,...,...,...,...,...,...
271435,2015-04-30 19:00:00,1036,48.0,43.25,43.25,39.943629,38.781402,38.328283,43.073911,48.0
271466,2015-04-30 20:00:00,1036,47.0,44.5,44.5,34.880253,35.281402,35.020101,44.261011,47.0
271497,2015-04-30 21:00:00,1036,NaN,47.0,47.0,42.504249,42.779354,43.558459,46.564303,NaN
271528,2015-04-30 22:00:00,1036,47.0,63.666667,63.666667,58.360097,52.920318,57.085251,60.193217,47.0


In [11]:
init = time()
feature = 'wind_speed'
    
out_df = run_k_folds()
print('time taken:', time()-init, 'seconds')
out_df

###
RMSE
###
linear 3.32546384108901
slinear 3.32546384108901
quadratic 4.29463665826604
cubic 4.504340861960426
spline 4.235280228258075
akima 3.4943989018580472
###
time taken: 12.01873779296875 seconds


,time,station_id,wind_speed,linear,slinear,quadratic,cubic,spline,akima,tmp_data
0,2014-05-01 00:00:00,1001,7.92,7.56,7.56,7.56,7.56,11.193251,7.56,NaN
31,2014-05-01 01:00:00,1001,7.56,6.84,6.84,6.18941,5.371294,7.58693,6.5,7.56
62,2014-05-01 02:00:00,1001,5.76,6.6,6.6,6.81091,6.792918,4.738994,6.662857,5.76
93,2014-05-01 03:00:00,1001,6.12,5.64,5.64,5.732124,5.779376,4.078732,5.651429,6.12
124,2014-05-01 04:00:00,1001,4.68,5.76,5.76,5.697546,5.521833,5.588553,5.729524,4.68
...,...,...,...,...,...,...,...,...,...,...
271435,2015-04-30 19:00:00,1036,2.60,2.925,2.925,2.528979,2.704657,2.592873,2.916671,2.60
271466,2015-04-30 20:00:00,1036,3.50,2.85,2.85,2.517696,2.634657,1.87204,2.850749,3.50
271497,2015-04-30 21:00:00,1036,NaN,3.1,3.1,3.335537,3.413799,2.186336,3.1,NaN
271528,2015-04-30 22:00:00,1036,2.70,2.9,2.9,3.623703,4.479443,2.456563,3.277011,2.70


## Imputing data with best methods

In [12]:
station_wise_df = []
best_methods = {'PM25_Concentration': 'akima', 
                'temperature': 'akima',
                'humidity': 'linear', 
                'wind_speed': 'linear',
                'weather': 'nearest',
                'wind_direction': 'nearest'}
for station in beijing_aq_plus_met2.station_id.unique():
    tmp_df = beijing_aq_plus_met2[beijing_aq_plus_met2.station_id == station]
    for var, method in best_methods.items():
        tmp_df[var] = tmp_df[var].interpolate(method).ffill().bfill()
    station_wise_df.append(tmp_df)
    
filled_df = pd.concat(station_wise_df)
filled_df

,time,station_id,longitude,latitude,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01 00:00:00,1001,116.173553,40.090679,138.000000,159.4,56.3,0.9,50.8,17.2,0.0,20.000000,56.0,7.92,13.0
31,2014-05-01 01:00:00,1001,116.173553,40.090679,124.000000,163.9,38.7,0.9,51.1,17.9,0.0,18.000000,64.0,7.56,13.0
62,2014-05-01 02:00:00,1001,116.173553,40.090679,127.000000,148.4,55.6,1.0,27.2,16.6,0.0,18.000000,70.0,5.76,13.0
93,2014-05-01 03:00:00,1001,116.173553,40.090679,129.000000,145.6,65.7,1.0,9.7,16.7,0.0,17.000000,74.0,6.12,13.0
124,2014-05-01 04:00:00,1001,116.173553,40.090679,119.000000,119.3,66.9,1.0,2.0,16.5,0.0,17.000000,75.0,4.68,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271435,2015-04-30 19:00:00,1036,116.000000,39.579999,77.000000,104.6,15.5,0.5,180.7,18.3,5.0,25.200000,48.0,2.60,23.0
271466,2015-04-30 20:00:00,1036,116.000000,39.579999,94.000000,141.2,26.1,0.6,146.1,16.9,5.0,25.000000,47.0,3.50,23.0
271497,2015-04-30 21:00:00,1036,116.000000,39.579999,96.064103,NaN,NaN,NaN,NaN,NaN,5.0,24.743628,47.0,3.10,23.0
271528,2015-04-30 22:00:00,1036,116.000000,39.579999,88.000000,NaN,23.0,0.7,120.6,15.8,5.0,24.400000,47.0,2.70,23.0


In [13]:
for var in best_methods.keys():
    assert filled_df[var].shape == filled_df[var].dropna().shape

In [14]:
final_df = filled_df[['time', 'station_id', 'longitude', 'latitude'] + list(best_methods.keys())]
final_df

,time,station_id,longitude,latitude,PM25_Concentration,temperature,humidity,wind_speed,weather,wind_direction
0,2014-05-01 00:00:00,1001,116.173553,40.090679,138.000000,20.000000,56.0,7.92,0.0,13.0
31,2014-05-01 01:00:00,1001,116.173553,40.090679,124.000000,18.000000,64.0,7.56,0.0,13.0
62,2014-05-01 02:00:00,1001,116.173553,40.090679,127.000000,18.000000,70.0,5.76,0.0,13.0
93,2014-05-01 03:00:00,1001,116.173553,40.090679,129.000000,17.000000,74.0,6.12,0.0,13.0
124,2014-05-01 04:00:00,1001,116.173553,40.090679,119.000000,17.000000,75.0,4.68,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
271435,2015-04-30 19:00:00,1036,116.000000,39.579999,77.000000,25.200000,48.0,2.60,5.0,23.0
271466,2015-04-30 20:00:00,1036,116.000000,39.579999,94.000000,25.000000,47.0,3.50,5.0,23.0
271497,2015-04-30 21:00:00,1036,116.000000,39.579999,96.064103,24.743628,47.0,3.10,5.0,23.0
271528,2015-04-30 22:00:00,1036,116.000000,39.579999,88.000000,24.400000,47.0,2.70,5.0,23.0


In [15]:
final_df.to_csv('../data/processed/final_data.csv.gz', index=None)